<a href="https://colab.research.google.com/github/kaniska/covid-19-hackathon/blob/master/social-network-mining/MedicalEntityRecognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%%time
!pip install pybind11
!pip install nmslib
!pip install scispacy spacy
!pip install googletrans
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.2.4/en_ner_bc5cdr_md-0.2.4.tar.gz
!pip install pytextrank

In [0]:
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.2.4/en_core_sci_sm-0.2.4.tar.gz
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.2.4/en_core_sci_md-0.2.4.tar.gz

In [0]:
import spacy
import scispacy
from scispacy.abbreviation import AbbreviationDetector
from scispacy.umls_linking import UmlsEntityLinker
#import en_core_web_sm
#import en_core_sci_sm
import en_ner_bc5cdr_md
import json
from collections import defaultdict, Counter
import pytextrank

In [0]:

#nlp1 = en_core_web_sm.load()
#nlp2 = en_core_sci_sm.load()
nlp3 = en_ner_bc5cdr_md.load()

linker = UmlsEntityLinker(resolve_abbreviations=True)
#nlp2.add_pipe(linker)

textRank = pytextrank.TextRank()
abbreviation_pipe = AbbreviationDetector(nlp3)
if not nlp3.has_pipe("abbrev"):
    nlp3.add_pipe(abbreviation_pipe, name="abbrev")
if not nlp3.has_pipe("textRank"):
    nlp3.add_pipe(textRank.PipelineComponent, name="textRank", last=True)
if not nlp3.has_pipe("linker"):
    nlp3.add_pipe(linker, name="linker")

In [0]:
from google.colab import drive
drive.mount('/content/drive')
with open(r"/content/drive/My Drive/TweetStreams/coronavirus-tweet-id-2020-04-10-21.jsonl", "r") as json_file:
    json_list = list(json_file)

In [0]:
for json_str in json_list:
    tweet_data = json.loads(json_str)    
    text_content = tweet_data['full_text']
    try:
        result = translator.translate(text_content)
        translatedText = result.text
        print(translatedText)
        if json_str and health_category(translatedText):
            print("Health Category Found")
            if covid_term_found(json_str):
                break
    except:
        print("Translation Failed")

In [0]:
def covid_term_found(tweet_text, tweet_id):
    #Approach 1
    doc1 = nlp3(tweet_text)
    #processed.append((tweet_data['id'], doc1))
    drugs = []
    diseases = []
    umls_entities = defaultdict(Counter)
    entities = defaultdict(lambda: defaultdict(Counter))
    
    entity_found = 0
    for entity in doc1.ents:
        # first find the entities and check the categories
        # 
        #if entity.label_  in ['CHEMICAL', 'DISEASE']:
        print("Tweet: ",json_str)
        print("Name: ", entity.text, "Label: ", entity.label_)
        entities[tweet_data['id']][entity.label_].update([entity])
        for umls_ent in entity._.umls_ents:
            print(linker.umls.cui_to_entity[umls_ent[0]])
            print(umls_ent[1])
            print("---------------------------------------------")
        # the umls candidate entities are attached to the entity mention, we'll take only the first
        top_candidate = next(iter([e for e in entity._.umls_ents if e[1] > 0.85]), None)
        if top_candidate:
            cuid = top_candidate[0]
            umls_entities[tweet_id].update([cuid])
            entity_found = 1
        #else:
        #    print("Medical term not found")

    for cuid, freq in umls_entities[tweet_id].most_common():
        entity = linker.umls.cui_to_entity[cuid]
        print(f"{freq} UID: {entity.concept_id}, name: {entity.canonical_name}, types: {entity.types}")

    
    if entity_found == 1:
        umls_entities[tweet_id]
        entities[tweet_id]
        return True
    
    return False

In [0]:
text = "coronavirus tests are going unused in US labs. Some can give results in 12-24 hours"
covid_term_found(text,123)


In [0]:
import argparse
import io
import json
import os
from google.cloud import language_v1
from google.cloud.language_v1 import enums

import numpy
import six

from google.cloud.language_v1 import types
import os
from google.oauth2 import service_account
import googletrans
from googletrans import Translator
translator = Translator()

from google.colab import drive
drive.mount('/content/drive')

language_client_v1 = language_v1.LanguageServiceClient.from_service_account_json("/content/drive/My Drive/secrets/niagra-a628a699be36.json")


In [0]:
def health_category(translatedText):
    #text = "Mulan Inspired merchandise available in China at Disney Store / Disney Town https://t.co/ZniMSngtwC"
    #text = "RT @JesusCintora: Nos hablan de que los socialcomunistas han propagado el covid, de una gestión criminal o de campos de concentración, pero…"
    document = language_v1.types.Document(
        content=translatedText,
        type=language_v1.enums.Document.Type.PLAIN_TEXT)

    entities = language_client_v1.analyze_entities(document)
    #show_entities(response1,5)
    for entity in entities.entities[:10]:
        if "Health" in entity.Type.Name(entity.type):
            print(' name: {0}'.format(entity.name))
            print(' type: {0}'.format(entity.Type.Name(entity.type)))
            print(' metadata: {0}'.format(dict(entity.metadata)))
            print(' salience: {:.4f}'.format(entity.salience))
            return True
        else:
            print(' name: {0}'.format(entity.name), ' type: {0}'.format(entity.Type.Name(entity.type)),' salience: {:.4f}'.format(entity.salience))
    return False


In [0]:
text = "coronavirus tests are going unused in US labs. Some can give results in 12-24 hours"
tmp1 = '"Contagion by covid-19 for doing their work as part of corresponding staff méd ...'
health_category(tmp1)

In [0]:
text = "Mulan Inspired merchandise available in China at Disney Store / Disney Town https://t.co/ZniMSngtwC"  
if health_category(text):
    print("Found Health Category")
else:
    print("Health Category Not Found")

In [0]:
def show_topic(response):
    for category in response.categories:
        # Get the name of the category representing the document.
        # See the predefined taxonomy of categories:
        # https://cloud.google.com/natural-language/docs/categories
        print(u"Category name: {}".format(category.name))
        # Get the confidence. Number representing how certain the classifier
        # is that this category represents the provided text.
        print(u"Confidence: {}".format(category.confidence))



In [0]:
def show_entities(entities, n):
	for entity in entities.entities[:n]:
		print('=' * 20)
		print(' name: {0}'.format(entity.name))
		print(' type: {0}'.format(entity.Type.Name(entity.type)))
		print(' metadata: {0}'.format(dict(entity.metadata)))
		print(' salience: {:.4f}'.format(entity.salience))

In [0]:
tmp1 = '"Contagion by covid-19 for doing their work as part of corresponding staff méd ...'
tmp2 = 'RT @JesusCintora: Nos hablan de que los socialcomunistas han propagado el covid, de una gestión criminal o de campos de concentración, pero'
result = translator.translate(tmp2)

print(result)